# Aula 07 - [Exemplos para oceanógrafos](https://ocefpaf.github.io/python4oceanographers/)

**Objetivos**

- Mostrar *code snippets* aplicados à oceanografia

### Exemplo 1: Planejamento de cruzeiro oceanográfico

In [ ]:
%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap


def make_map(lonStart=-48, lonEnd=-32, latStart=-30, latEnd=-18, img=None):
    m = Basemap(projection='merc', llcrnrlon=lonStart, urcrnrlon=lonEnd,
                llcrnrlat=latStart, urcrnrlat=latEnd, resolution='c',
                lat_ts=(latStart + latEnd) / 2.)

    fig, ax = plt.subplots(figsize=(6, 6), facecolor='w')
    m.ax = ax

    image = plt.imread(img)
    m.imshow(image, origin='upper', alpha=0.75)


    lonStart, latStart = -42, -26  # Crop the image.
    lon_lim, lat_lim = m([lonStart, lonEnd], [latStart, latEnd])
    m.ax.axis([lon_lim[0], lon_lim[1], lat_lim[0], lat_lim[1]])

    dx = dy = 2
    meridians = np.arange(lonStart, lonEnd + dy, dy)
    parallels = np.arange(latStart,  latEnd + dx, dx)
    xoffset = -lon_lim[0] + 1e4
    yoffset = -lat_lim[0] + 1e4
    kw = dict(linewidth=0)
    m.drawparallels(parallels, xoffset=xoffset, labels=[1, 0, 0, 0], **kw)
    m.drawmeridians(meridians, yoffset=yoffset, labels=[0, 0, 0, 1], **kw)
    return fig, m

In [ ]:
lon = [-40.77, -40.51, -40.30, -40.23, -40.13, -40.06, -39.99,
       -39.87, -39.72, -39.52, -39.32, -39.11, -38.91, -38.71]
lat = [-21.29, -21.39, -21.48, -21.51, -21.56, -21.58, -21.62,
       -21.69, -21.76, -21.86, -21.96, -22.08, -22.15, -22.25]

fig, m = make_map(img='./data/AVHRR.png')
kw = dict(marker='o', markerfacecolor='k', markeredgecolor='w', markersize=6, linestyle='none')
_ = m.plot(*m(lon, lat), **kw)

### Exemplo 2: Análise exploratória

In [ ]:
import seaborn
import numpy as np
import matplotlib.pyplot as plt

from io import BytesIO
from pandas import read_csv


kw = dict(na_values='NaN', sep=',', encoding='utf-8',
          skipinitialspace=True, index_col=False)

df = read_csv("./data/fish.csv", **kw)
df.head()

In [ ]:
kw = {'axes.edgecolor': '0', 'text.color': '0', 'ytick.color': '0', 'xtick.color': '0',
      'ytick.major.size': 5, 'xtick.major.size': 5, 'axes.labelcolor': '0'}

seaborn.set_style("whitegrid", kw)

ax = seaborn.corrplot(df, annot=False, diag_names=False)

In [ ]:
g = df.groupby('Days')

mean_df = g.mean()

g.describe().head(16)

In [ ]:
ax = seaborn.jointplot("Days", "BDE 99 (ng/g)", df, kind="reg")

In [ ]:
ax = seaborn.jointplot("Days", "BDE 47 (ng/g)", df, kind="reg")

In [ ]:
ax = seaborn.residplot("Days", "BDE 47 (ng/g)", df)

### Exemplo 3: Ler dados em NetCDF CF-1.6 de CTD

In [ ]:
import iris

url = 'http://data.nodc.noaa.gov/thredds/dodsC/testdata/netCDFTemplateExamples/profile/wodStandardLevels.nc'
url = 'wodStandardLevels.nc'
sal = iris.load_cube(url, 'sea_water_salinity')

print(sal)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from cartopy.feature import LAND, COASTLINE
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

x = sal.coord('longitude').points
y = sal.coord('latitude').points

fig, ax = plt.subplots(figsize=(11, 13),
                       subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.set_extent([-180, 180, -90, 90])
ax.stock_img()
ax.plot(x, y, 'go')

ax.add_feature(COASTLINE)
kw = dict(linewidth=1.5, color='gray', alpha=0.5, linestyle='--')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, **kw)
gl.xlabels_top = gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

from shapely.geometry.polygon import LinearRing
lons = [105, 105, 125, 125]
lats = [-40, -20, -20, -40]
ring = LinearRing(list(zip(lons, lats)))
sq = ax.add_geometries([ring], ccrs.PlateCarree(), facecolor='none', edgecolor='red')

In [ ]:
lon = iris.Constraint(longitude=lambda l:min(lons) < l < max(lons))
lat = iris.Constraint(latitude=lambda l:min(lats) < l < max(lats))
alt = iris.Constraint(altitude=lambda a: a <= 30)

sal_profile = sal.extract(alt & lon & lat)

print(sal_profile)

In [ ]:
import iris.plot as iplt

lon = sal_profile.coord(axis='X').points.squeeze()
lat = sal_profile.coord(axis='Y').points.squeeze()
depth = sal_profile.coord(axis='Z').points.max()

fig, ax = plt.subplots(figsize=(5, 6))
kw = dict(linewidth=2,  color='k', marker='o')
iplt.plot(sal_profile, sal_profile.coord('altitude'), **kw)
ax.grid()
ax.set_ylabel('Depth (m)')
ax.set_xlabel('Salinity (unknown)')
t = ax.set_title('lon: %s\nlat: %s\nMax depth = %s' % (lon, lat, depth))

### Exemplo 4: PCA

In [ ]:
import numpy as np
from pandas import DataFrame

x = np.array([2.5, 0.5, 2.2, 1.9, 3.1, 2.3, 2.0, 1.0, 1.5, 1.1])
y = np.array([2.4, 0.7, 2.9, 2.2, 3.0, 2.7, 1.6, 1.1, 1.6, 0.9])

Z = np.c_[x - x.mean(), y - y.mean()]
df = DataFrame(Z, columns=['x', 'y'])
df.index.name = 'Medidas'
df.T

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

line = dict(linewidth=1, linestyle='--', color='k')
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)


fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(x, y, **marker)
ax.axhline(**line)
ax.axvline(**line)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(u'Dados Originais (com a média)')
_ = ax.axis([-1, 4, -1, 4])

In [ ]:
cov = np.cov(Z.T)
cov

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov)
eigenvalues

In [ ]:
eigenvectors

In [ ]:
pc = eigenvectors[1] / eigenvectors[0]
pc

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(Z[:, 0], Z[:, 1], **marker)
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.axhline(**line)
ax.axvline(**line)
ax.set_xlabel('x')
ax.set_ylabel('y')

arrowprops = dict(width=0.01, head_width=0.05, alpha = 0.5,
                  length_includes_head=False)
a1 = ax.arrow(0, 0, eigenvalues[0], pc[0] * eigenvalues[0],
              color='k', **arrowprops)
a2 = ax.arrow(0, 0, eigenvalues[1], pc[1] * eigenvalues[1],
              color='k', **arrowprops)
a3 = ax.arrow(0, 0, -eigenvalues[0], -pc[0] * eigenvalues[0],
              color='r', **arrowprops)
a4 = ax.arrow(0, 0, -eigenvalues[1], -pc[1] * eigenvalues[1],
              color='r', **arrowprops)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, copy=True)
X = pca.fit_transform(Z)

fig, ax = plt.subplots()
ax.plot(X[:, 0], X[:, 1], **marker) 
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.axhline(**line)
ax.axvline(**line)
_ = ax.set_title("PCA")

In [ ]:
print(eigenvectors)
print('')
print(pca.components_)

Quanto da variância é explicada por cada componente.

In [ ]:
pca.explained_variance_ratio_

In [ ]:
from scipy.stats.stats import pearsonr

r, p = pearsonr(x-x.mean(), y-y.mean())

print('Pearson r: {}\nPC 1: {}:'.format(r, pc[0]))

In [ ]:
import statsmodels.api as sm


def OLSreg(y, Xmat):
    return sm.OLS(y, sm.add_constant(Xmat, prepend=True)).fit()


scale = lambda x: (x - x.mean()) / x.std()

ols_fit = OLSreg(Z[:, 1], Z[:, 0])

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(Z[:, 0], Z[:, 1], **marker)
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.axhline(**line)
ax.axvline(**line)
ax.set_xlabel('x')
ax.set_xlabel('y')

ax.plot(Z[:, 0], ols_fit.fittedvalues, 'r', alpha=0.5)
ax.text(-1, 1, r'R$^2$ = %4.3f' % round(ols_fit.rsquared, 3))

a1 = ax.arrow(0, 0, eigenvalues[1], pc[1] * eigenvalues[1],
              **arrowprops)
a2 = ax.arrow(0, 0, -eigenvalues[1], -pc[1] * eigenvalues[1],
              **arrowprops)

### Dados reais (podemos tentar com dados dos alunos)

In [ ]:
%%file data.csv
Food,England,Wales,Scotland,N Ireland
Cheese,105,103,103,66
Carcass meat,245,227,242,267
Other meat,685,803,750,586
Fish,147,160,122,93
Fats and oils,193,235,184,209
Sugars,156,175,147,139
Fresh potatoes,720,874,566,1033
Fresh Veg,253,265,171,143
Other Veg,488,570,418,355
Processed potatoes,198,203,220,187
Processed Veg,360,365,337,334
Fresh fruit,1102,1137,957,674
Cereals,1472,1582,1462,1494
Beverages,57,73,53,47
Soft drinks,1374,1256,1572,1506
Alcoholic drinks,375,475,458,135
Confectionery,54,64,62,41

In [ ]:
from pandas import read_csv

df = read_csv('data.csv', index_col='Food')
df

In [ ]:
def z_score(x):
    """Remove a média e normaliza os pelo desvio padrão"""
    return (x - x.mean()) / x.std()

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=None)
pca.fit(df.apply(z_score).T)

In [ ]:
from pandas import DataFrame

loadings = DataFrame(pca.components_.T)
loadings.index = ['PC %s' % pc for pc in loadings.index + 1]
loadings.columns = ['TS %s' % pc for pc in loadings.columns + 1]
loadings

In [ ]:
PCs = np.dot(loadings.values.T, df)

In [ ]:
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)

fig, ax = plt.subplots(figsize=(7, 2.75))
ax.plot(PCs[0], np.zeros_like(PCs[0]),
        label="Scores", **marker)
[ax.text(x, y, t) for x, y, t in zip(PCs[0], loadings.values[1, :], df.columns)]

ax.set_xlabel("PC1")

_ = ax.set_ylim(-1, 1)
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 2.75))
ax.plot(PCs[0], PCs[1], label="Scores", **marker)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")

text = [ax.text(x, y, t) for x, y, t in
        zip(PCs[0], PCs[1]+0.5, df.columns)]

In [ ]:
perc = pca.explained_variance_ratio_ * 100

perc = DataFrame(perc, columns=['Percentage explained ratio'], index=['PC %s' % pc for pc in np.arange(len(perc)) + 1])
ax = perc.plot(kind='bar')

In [ ]:
TS1 = loadings['TS 1']
TS1.index = df.index
ax = TS1.plot(kind='bar')

In [ ]:
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)

fig, ax = plt.subplots(figsize=(7, 7))
ax.plot(loadings.icol(0), loadings.icol(1), label="Loadings", **marker)
ax.set_xlabel("non-projected PC1")
ax.set_ylabel("non-projected PC2")
ax.axis([-1, 1, -1, 1])
text = [ax.text(x, y, t) for
        x, y, t in zip(loadings.icol(0), loadings.icol(1), df.index)]

### Exemplo 5: SymPy

In [ ]:
from sympy.interactive import printing
from sympy import Function, symbols, sqrt, S

printing.init_printing()
x, y = symbols("x y")

In [ ]:
from IPython.display import Latex

red = S("(x / 7) ** 2 * sqrt(abs(abs(x) - 3) / (abs(x) - 3)) + (y / 3) ** 2 * sqrt(abs(y + 3 / 7 * sqrt(33)) / (y + 3 / 7 * sqrt(33))) - 1")

red

In [ ]:
orange = S("abs(x / 2) - ((3 * sqrt(33) - 7) / 112) * x ** 2 - 3 + sqrt(1 - (abs(abs(x) - 2) - 1) ** 2) - y")
orange

In [ ]:
green = S("9 * sqrt(abs((abs(x) - 1) * (abs(x) - 0.75)) / ((1 - abs(x)) * (abs(x) - 0.75))) - 8 * abs(x) - y")
green

In [ ]:
blue = S("3 * abs(x) + 0.75 * sqrt(abs((abs(x) - 0.75) * (abs(x) - 0.5)) / ((0.75 - abs(x)) * (abs(x) - 0.5))) - y")
blue

In [ ]:
pink = S("2.25 * sqrt(abs((x - 0.5) * (x + 0.5)) / ((0.5 - x) * (0.5 + x))) - y")
pink

In [ ]:
brown = S("6 * sqrt(10) / 7 + (1.5 - 0.5 * abs(x)) * sqrt(abs(abs(x) - 1) / (abs(x) - 1)) - (6 * sqrt(10) / 14) * sqrt(4 - (abs(x) - 1) ** 2) - y")
brown

In [ ]:
from sympy import lambdify

def lamb(func):
    return lambdify((x, y), func, modules='numpy')

red, orange, green, blue, pink, brown = map(lamb, (red, orange, green, blue, pink, brown))

In [ ]:
import numpy as np

spacing = 0.01
x, y = np.meshgrid(np.arange(-7.25, 7.25, spacing),
                   np.arange(-5, 5, spacing))

red = red(x, y)
orange = orange(x, y)
green = green(x, y)
blue = blue(x, y)
pink = pink(x, y)
brown = brown(x, y)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

colors = dict(red='#FF0000', orange='#FFA500', green='#008000',
              blue='#003399', pink='#CC0033', brown='#800000')

equations = dict(red=red, orange=orange, green=green,
                 blue=blue, pink=pink, brown=brown)
                 
fig, ax = plt.subplots(figsize=(8, 6))
for key, color in colors.items():
    ax.contour(x, y, equations[key], [0], colors=color, linewidths=3)
    
_ = ax.set_xticks([])
_ = ax.set_yticks([])